In [ ]:
%matplotlib widget

import os
import pathlib
import pprint
import numpy as np

from Visualizer.DataSet import DataSet
from Visualizer.Monitoring.Persistence.ProfilerSession import ProfilerSession
from Visualizer.Plotting.Plot import Plot

In [ ]:
source_directory = pathlib.Path().parent.absolute()
project_directory = f"{source_directory}/.."
root_directory = f"{project_directory}/.."
resources_directory = f"{root_directory}/Resources"
database = f"{resources_directory}/Database.sqlite"

# Load all profiler sessions
complete_data_set = DataSet(ProfilerSession.load_all(database))

In [ ]:
overview_plot = complete_data_set.table_plot
overview_plot.maximum_rows = 500
overview_plot.plot()

In [ ]:
begin = 109
num_of_policies = 3
num_of_profiles = 1
iterations = 5
energy_dict = {}
timespan_dict = {}

for profiler_session_id in range(begin, begin + num_of_policies*num_of_profiles*iterations,  num_of_profiles):
    data_set = DataSet([profiler_session for profiler_session in complete_data_set.data if profiler_session.id == profiler_session_id])
    
    energy_consumption = next(reversed(data_set.data[0].energy_consumption()['Node'].values()))
    timespan = next(reversed(data_set.data[0].timespan()['Runtime'].values()))
    
    label = data_set.data[0].summary['Label']
    if not label in energy_dict:
        energy_dict[label] = []
        timespan_dict[label] = []
    if energy_consumption > 0:
        energy_dict[label].append(energy_consumption)
    timespan_dict[label].append(timespan)

    
    print(profiler_session_id, label, energy_consumption, timespan)

In [ ]:
print(energy_dict)
avg_energy_list = [np.mean(energy_dict[key]) for key in energy_dict]
print(avg_energy_list)
energy_percentage_list =[(avg_energy-avg_energy_list[0])/avg_energy_list[0]*100 for avg_energy in avg_energy_list]
print(energy_percentage_list)
    

In [ ]:
print(timespan_dict)
avg_timespan_list = [np.mean(timespan_dict[key]) for key in timespan_dict]
print(avg_timespan_list)
timespan_percentage_list =[(avg_timespan-avg_timespan_list[0])/avg_timespan_list[0]*100 for avg_timespan in avg_timespan_list]
print(timespan_percentage_list)